In [21]:
import numpy as np
import random 
from scipy.stats import unitary_group
# defining Pauli matrices
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

# defining rotation gates
def Ry(A):
    M = np.array([[np.cos(A/2),-np.sin(A/2)],[np.sin(A/2), np.cos(A/2)]])
    return M
def Rz(A):
    M = np.array([[np.exp(-1j*A/2),0],[0,np.exp(1j*A/2)]])
    return M
    
# Parameterising the general unitary with rotation gates
def U (A0,A1,A2): # Arguments: Euler Angles
    M = Rz(A1)@Ry(A0)@Rz(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

# generate random unitary 
def Urand():
    A0=random.uniform(0.0, 4*np.pi)
    A1=random.uniform(0.0, 4*np.pi)
    A2=random.uniform(0.0, 4*np.pi)
    Urand= U(A0,A1,A2)
    return Urand

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse



In [22]:
# function to define RHS for each qubit
def Utbm(j, A):
    U1 = U(A[0,0],A[0,1],A[0,2])
    U2 = U(A[1,0],A[1,1],A[1,2])
    U3 = U(A[2,0],A[2,1],A[2,2])
    U4 = U(A[3,0],A[3,1],A[3,2])
    U5 = U(A[4,0],A[4,1],A[4,2])
    U6 = U(A[5,0],A[5,1],A[5,2])
    U7 = U(A[6,0],A[6,1],A[6,2])
    U8 = U(A[7,0],A[7,1],A[7,2])
    Ulist = [U1,U2,U3,U4,U5,U6,U7,U8]
    Ulist[j-1] = np.eye(2)
    U12 = np.kron(Ulist[0],Ulist[1])
    U34 = np.kron(Ulist[2],Ulist[3])
    U56 = np.kron(Ulist[4],Ulist[5])
    U78 = np.kron(Ulist[6],Ulist[7])
    if j==1:
        Uf = np.eye(4)
        Ub = U12@CNOT@U34@CNOT@U56@CNOT@U78
    elif j==2:
        Uf = U12
        Ub = CNOT@U34@CNOT@U56@CNOT@U78
    elif j==3:
        Uf = U12@CNOT
        Ub =U34@CNOT@U56@CNOT@U78
    elif j==4:
        Uf = U12@CNOT@U34
        Ub =CNOT@U56@CNOT@U78
    elif j==5:
        Uf = U12@CNOT@U34@CNOT
        Ub = U56@CNOT@U78
    elif j==6:
        Uf = U12@CNOT@U34@CNOT@U56
        Ub = CNOT@U78
    elif j==7:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT
        Ub = U78
    elif j==8:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT@U78
        Ub = np.eye(4)
    U_tbm = inverse(Uf)@U_goal@inverse(Ub)
    return U_tbm

In [30]:
def gradient_descent(A, iterations,alpha):
    Aopt = np.copy(A)
    for j in range(1,9):
        Ui = Utbm(j,Aopt)
        A0=A[j-1][0]
        A1=A[j-1][1]
        A2=A[j-1][2]
        #print("initial:",A0,A1,A2)
        U_initial= U(A0,A1,A2)
        if j%2!=0:
            Uxyz = np.kron(U_initial,np.eye(2))
            #print(Uxyz)
            #print(j)
        else:
            Uxyz = np.kron(np.eye(2), U_initial)
            #print(Uxyz)
            #print(j)
        for i in range(iterations):
            Tr = np.trace(Ui@inverse(Uxyz))
            Abs = np.abs(Tr)
            Uerr = 4 - Abs
            #print(Abs/Tr)
            print(Uerr)
            if Uerr <= 1e-8:
                break
            else:
                dA0 = DA0(A0,A1,A2)
                dA1 = DA1(A0,A1,A2)
                dA2 = DA2(A0,A1,A2)
                if j%2!=0:
                    UdA0 = np.kron(dA0,np.eye(2))
                    UdA1 = np.kron(dA1,np.eye(2))
                    UdA2 = np.kron(dA2,np.eye(2))
                else:
                    UdA0 = np.kron(np.eye(2),dA0)
                    UdA1 = np.kron(np.eye(2),dA1)
                    UdA2 = np.kron(np.eye(2),dA2)
                Grad_A0 = -np.conj((Abs/Tr))*(np.trace(Ui@inverse(UdA0)))
                Grad_A0R = Grad_A0.real
                Grad_A1 = -np.conj((Abs/Tr))*(np.trace(Ui@inverse(UdA1)))
                Grad_A1R = Grad_A1.real
                Grad_A2 = -np.conj((Abs/Tr))*(np.trace(Ui@inverse(UdA2)))
                Grad_A2R = Grad_A2.real
                A0 = A0 - alpha*Grad_A0R
                A1 = A1 - alpha*Grad_A1R
                A2 = A2 - alpha*Grad_A2R
                #print(Grad_A0, Grad_A1, Grad_A2)
                if j%2 != 0:
                    Uxyz = np.kron(U(A0,A1,A2),np.eye(2))
                else:
                    Uxyz = np.kron(np.eye(2), U(A0,A1,A2) )
            #print(Uxyz)
        Aopt[j-1][0]=A0
        Aopt[j-1][1]=A1
        Aopt[j-1][2]=A2
        #print(Aopt)
    return Aopt

In [31]:
CNOT = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]] # switch tensor control

# goal unitary
U_goal = unitary_group.rvs(4)

A = np.zeros((8,3))
for i in range(8):
    for j in range(3):
        A[i][j] = random.uniform(0.0, 4*np.pi) 


AOpt =np.copy(A)
for i in range(100):
    AOpt = gradient_descent(AOpt,10,0.01)

#print("AOpt =",AOpt)

#A_result = gradient_descent(U_tbm,1000,0.01)

#verification
U_1 = U(AOpt[0][0],AOpt[0][1],AOpt[0][2])
U_2 = U(AOpt[1][0],AOpt[1][1],AOpt[1][2])
U_3 = U(AOpt[2][0],AOpt[2][1],AOpt[2][2])
U_4 = U(AOpt[3][0],AOpt[3][1],AOpt[3][2])
U_5 = U(AOpt[4][0],AOpt[4][1],AOpt[4][2])
U_6 = U(AOpt[5][0],AOpt[5][1],AOpt[5][2])
U_7 = U(AOpt[6][0],AOpt[6][1],AOpt[6][2])
U_8 = U(AOpt[7][0],AOpt[7][1],AOpt[7][2])


U_12 = np.kron(U_1,U_2)
U_34 = np.kron(U_3,U_4)
U_56 = np.kron(U_5,U_6)
U_78 = np.kron(U_7,U_8)
U_found = U_12@CNOT@U_34@CNOT@U_56@CNOT@U_78


3.8489630342659273
3.847700565890834
3.846119915776302
3.8442492154748424
3.8421175667524543
3.8397540035619517
3.8371866602396296
3.834442160375327
3.83154521382855
3.8285183928041033
3.8253820506392384
3.824630754021013
3.82386953232353
3.823098902893299
3.822319369320679
3.821531421263369
3.820735534330944
3.8199321700254094
3.819121775732876
3.818304784761624
3.817481616422024
3.814332183181554
3.8112345693556002
3.8081876510232213
3.805190155184055
3.802240714894295
3.799337910355683
3.796480299377914
3.7936664397921604
3.790894905763058
3.7881642994740132
3.788076998170457
3.787982781108854
3.7878819285063385
3.7877747132281505
3.7876614008260483
3.7875422495888724
3.787417510604242
3.787287427830411
3.7871522381773444
3.7870121715961127
3.781684902224083
3.7762981933427406
3.7708634492764896
3.7653902702828566
3.7598867387290524
3.7543596625627766
3.7488147809149375
3.7432569368378403
3.7376902219416515
3.732118097249569
3.731832924181258
3.731548317055734
3.7312643190370576
3.7

1. How fast does it converge
2. What's a stable learning rate
3. Does it reliably converge
4. Compare different unitaries and their convergence rate

In [32]:
print(U_goal)
print(U_found)

[[ 0.50935505-0.12489443j -0.31118536+0.32328195j -0.37294959+0.03554568j
   0.61314548+0.08549268j]
 [ 0.34712834-0.23679143j  0.17851972+0.30859926j -0.42126283-0.10307703j
  -0.68601433+0.19397432j]
 [-0.6099003 +0.25901685j -0.26350194+0.65966858j -0.1596601 +0.08990341j
  -0.04454962+0.14414351j]
 [-0.17865831+0.27821514j  0.04885064-0.40566903j -0.77127704+0.20764301j
   0.00342262-0.29279377j]]
[[ 0.62628095+0.19141058j -0.3094104 +0.08086994j -0.36817158-0.04043648j
   0.45824427+0.34883547j]
 [ 0.28545014-0.1703143j  -0.02084597+0.29370621j -0.44334829-0.43080327j
  -0.60875836-0.22377992j]
 [-0.58694774-0.06090693j -0.59998035+0.48214968j -0.14818224-0.09739426j
   0.13639233+0.09639234j]
 [-0.32341212+0.08850265j  0.15278948-0.44185745j -0.65408452-0.14627108j
   0.31325408-0.34876832j]]


In [3]:
# Generating random unitary
x =unitary_group.rvs(4)
a = np.dot(x,inverse(x))
print(x)

NameError: name 'unitary_group' is not defined